Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
from __future__ import print_function
import math
import os

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
# DATA_DIR = '/Users/gleb/Data/udacity-tensorflow-data'
DATA_DIR = './'

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = os.path.join(DATA_DIR, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
            shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Define Variables.
    
    with tf.name_scope('hidden1'):
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]),
            name='weights')
        biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]), name='biases')
        
    with tf.name_scope('output_linear'):
        weights_2 = tf.Variable(
            tf.truncated_normal([hidden_1_num_nodes, num_labels]),
            name='weights')
        biases_2 = tf.Variable(tf.zeros([num_labels]), name='biases')
    
    def build_network(input_data):
        """Function that builds the network, allowing the code to be used by
        training, validationg, and test.

        Returns handle to the output layer.
        """
        hidden_1 = tf.nn.relu(
                tf.matmul(input_data, weights_1) + biases_1)
        logits = tf.matmul(hidden_1, weights_2) + biases_2
        return logits

    # Training computation.
    logits = build_network(tf_train_dataset)
    
    # Calculate loss.
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Add L2 regularization.
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(build_network(tf_valid_dataset))

    test_prediction = tf.nn.softmax(build_network(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 503.235168
Minibatch accuracy: 7.8%
Validation accuracy: 28.6%
Minibatch loss at step 500: 132.569183
Minibatch accuracy: 75.8%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 98.815147
Minibatch accuracy: 79.7%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 74.575134
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 57.319382
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 44.097458
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 34.699703
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
batch_size = 128

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
            shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Define Variables.
    
    with tf.name_scope('hidden1'):
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]),
            name='weights')
        biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]), name='biases')
        
    with tf.name_scope('output_linear'):
        weights_2 = tf.Variable(
            tf.truncated_normal([hidden_1_num_nodes, num_labels]),
            name='weights')
        biases_2 = tf.Variable(tf.zeros([num_labels]), name='biases')
    
    def build_network(input_data):
        """Function that builds the network, allowing the code to be used by
        training, validationg, and test.

        Returns handle to the output layer.
        """
        hidden_1 = tf.nn.relu(
                tf.matmul(input_data, weights_1) + biases_1)
        logits = tf.matmul(hidden_1, weights_2) + biases_2
        return logits

    # Training computation.
    logits = build_network(tf_train_dataset)
    
    # Calculate loss.
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Add L2 regularization.
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(build_network(tf_valid_dataset))

    test_prediction = tf.nn.softmax(build_network(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Always same offset to force over-fitting.
    offset = 0

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 488.964294
Minibatch accuracy: 12.5%
Validation accuracy: 33.1%
Minibatch loss at step 500: 122.521378
Minibatch accuracy: 100.0%
Validation accuracy: 62.0%
Minibatch loss at step 1000: 95.416756
Minibatch accuracy: 100.0%
Validation accuracy: 62.0%
Minibatch loss at step 1500: 74.308334
Minibatch accuracy: 100.0%
Validation accuracy: 62.0%
Minibatch loss at step 2000: 57.869568
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 2500: 45.067463
Minibatch accuracy: 100.0%
Validation accuracy: 62.1%
Minibatch loss at step 3000: 35.097500
Minibatch accuracy: 100.0%
Validation accuracy: 62.2%
Test accuracy: 68.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
            shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Define Variables.
    
    with tf.name_scope('hidden1'):
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]),
            name='weights')
        biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]), name='biases')
        
    with tf.name_scope('output_linear'):
        weights_2 = tf.Variable(
            tf.truncated_normal([hidden_1_num_nodes, num_labels]),
            name='weights')
        biases_2 = tf.Variable(tf.zeros([num_labels]), name='biases')
        
    keep_prob = tf.placeholder(tf.float32)
    
    def build_network(input_data):
        """Function that builds the network, allowing the code to be used by
        training, validationg, and test.

        Returns handle to the output layer.
        """
        hidden_1 = tf.nn.relu(
                tf.matmul(input_data, weights_1) + biases_1)
        hidden_1_dropout = tf.nn.dropout(hidden_1, keep_prob)
        logits = tf.matmul(hidden_1_dropout, weights_2) + biases_2
        return logits

    # Training computation.
    logits = build_network(tf_train_dataset)
    
    # Calculate loss.
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Add L2 regularization.
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(build_network(tf_valid_dataset))

    test_prediction = tf.nn.softmax(build_network(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Always same offset to force over-fitting.
        # offset = 0

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels : batch_labels,
            keep_prob: 0.5
        }

        # Run an evaluation step.
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        # Sample the current strength of the neural net using the validation set.
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    
    # Output the final test prediction.  
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 619.902283
Minibatch accuracy: 10.9%
Validation accuracy: 21.3%
Minibatch loss at step 500: 141.884644
Minibatch accuracy: 60.2%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 103.140160
Minibatch accuracy: 75.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 74.981789
Minibatch accuracy: 73.4%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 58.980354
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 44.057259
Minibatch accuracy: 74.2%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 34.035896
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Test accuracy: 89.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [37]:
batch_size = 128

hidden_1_num_nodes = 1024

hidden_2_num_nodes = 300

hidden_3_num_nodes = 50

INITIAL_LEARNING_RATE = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    keep_prob = tf.placeholder(tf.float32)
    
    weights_1 = tf.Variable(
        tf.truncated_normal(
                [image_size * image_size, hidden_1_num_nodes],
                stddev=1.0
                # stddev=1.0 / math.sqrt(float(image_size * image_size))
        )
    )
    biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]))
    
    weights_hidden_1_out = tf.Variable(
        tf.truncated_normal(
                [hidden_1_num_nodes, hidden_2_num_nodes],
                stddev=1.0
        )
    )
    biases_hidden_1_out = tf.Variable(tf.zeros([hidden_2_num_nodes]))
    
    weights_hidden_2_out = tf.Variable(
        tf.truncated_normal(
                [hidden_2_num_nodes, hidden_3_num_nodes],
                stddev=1.0
        )
    )
    biases_hidden_2_out = tf.Variable(tf.zeros([hidden_3_num_nodes]))
    
    weights_hidden_3_out = tf.Variable(
        tf.truncated_normal(
                [hidden_3_num_nodes, num_labels],
                stddev=1.0
        )
    )
    biases_hidden_3_out = tf.Variable(tf.zeros([num_labels]))
    
    def build_network(input_data):
        """Function that builds the core inner network.

        Allows making sure that train, validate, and test are really
        using the same network.

        Returns handle to the output layer.
        """
        hidden_1 = tf.nn.relu(
                tf.matmul(input_data, weights_1) + biases_1)
        hidden_1_dropout = tf.nn.dropout(hidden_1, keep_prob)
            
        hidden_2 = tf.nn.relu(
                tf.matmul(hidden_1_dropout, weights_hidden_1_out) + 
                biases_hidden_1_out)
        hidden_2_dropout = tf.nn.dropout(hidden_2, keep_prob)
        
        hidden_3 = tf.nn.relu(
                tf.matmul(hidden_2_dropout, weights_hidden_2_out) + 
                biases_hidden_2_out)
        hidden_3_dropout = tf.nn.dropout(hidden_3, keep_prob)
        
        logits = (
                tf.matmul(hidden_3_dropout, weights_hidden_3_out) +
                biases_hidden_3_out)

        return logits

    # Training computation.
    logits = build_network(tf_train_dataset)

    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                    logits, tf_train_labels))
    
    #   Add regularization for training.
#     reg_constant = 5e-3
#     reg = reg_constant * (
#             tf.nn.l2_loss(weights_1) +
#             tf.nn.l2_loss(weights_hidden_1_out) +
#             tf.nn.l2_loss(weights_hidden_2_out) +
#             tf.nn.l2_loss(weights_hidden_3_out)
#     )
#     loss += reg
    
    # Optimizer with decaying learning rate.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
            INITIAL_LEARNING_RATE, global_step, 100, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
            loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_prediction = tf.nn.softmax(build_network(tf_valid_dataset))

    test_prediction = tf.nn.softmax(build_network(tf_test_dataset))

num_steps = 5001

with tf.Session(graph=graph) as session:
    log_path = os.path.join(DATA_DIR, 'log')
    summary_writer = tf.train.SummaryWriter(log_path, session.graph)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels : batch_labels,
            keep_prob: 0.9
        }

        # Run an evaluation step.
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        # Sample the current strength of the neural net using the validation set.
        if (step % 500 == 0):
            print("Global step: %d" % global_step.eval())
            print("Learning rate: %f" % learning_rate.eval())
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    
    # Output the final test prediction.  
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Global step: 1
Learning rate: 0.009996
Minibatch loss at step 0: 22034.181641
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Global step: 501
Learning rate: 0.008150
Minibatch loss at step 500: 2.302114
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Global step: 1001
Learning rate: 0.006646
Minibatch loss at step 1000: 2.302581
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Global step: 1501
Learning rate: 0.005419
Minibatch loss at step 1500: 2.302729
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Global step: 2001
Learning rate: 0.004418
Minibatch loss at step 2000: 74.387444
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Global step: 2501
Learning rate: 0.003602
Minibatch loss at step 2500: 2.302676
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Global step: 3001
Learning rate: 0.002937
Minibatch loss at step 3000: 2.302577
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Global step: 3501
Learning rate: 0.002395
Minibatch loss at step 